### 1. 🧪 Install Required Libraries (Run only once)

In [ ]:


# !pip install -q langchain faiss-cpu python-dotenv pypdf unstructured huggingface_hub transformers

# Optional: For local models with Ollama (skip if using Hugging Face only)
# !pip install langchain-community


Defaulting to user installation because normal site-packages is not writeable


### 2. 🔐 Load Environment Variables (from .env)

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()
HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")


### 3. 📚 Load Your LOTR PDF (Adjust path as needed)

In [3]:


from langchain.document_loaders import PyPDFLoader

pdf_path = "Tolkien-J.-The-lord-of-the-rings-HarperCollins-ebooks-2010.pdf"
loader = PyPDFLoader(pdf_path)
pages = loader.load()

print(f"Loaded {len(pages)} pages from PDF.")


Loaded 1210 pages from PDF.


### 4. ✂️ Chunk the Text for Embedding

In [4]:


from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

docs = text_splitter.split_documents(pages)
print(f"Total chunks: {len(docs)}")


Total chunks: 4246


### 5. 🔎 Create FAISS Vector Store with Embeddings

In [7]:
# !pip install -U langchain-huggingface sentence-transformers

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/204.1 MB ? eta -:--:--
   ---------------------------------------- 1.3/204.1 MB 7.5 MB/s eta 0:00:27
    --------------------------------------- 3.9/204.1 MB 10.2 MB/s eta 0:00:20
   - -------------------------------------- 7.1/204.1 MB 11.9 MB/s eta 0:00:17
   -- ------------------------------------- 10.2/204.1 MB 12.8 MB/s eta 0:00:16
   -- ------------------------------------- 13.4/204.1 MB 13.2 MB/s eta 0:00:15
   --- ------------------------------------ 16.3/204.1 MB 13.5 MB/s eta 0:00:14
   --- ------------------------------------ 19.4/204.1 MB 13.7 MB/s eta 0:00:14
   ---- ----------------------------------- 22.5/204.1 MB 13.9 MB/s eta 0:00:14
   ----- ---------------------------------- 25.7/204.1 MB 14.0 MB/s eta 0:00:13
   ----- ---------------------------------- 29.1/204.1 MB 14.1 MB/s eta 0:00:13
   ------ --------------------------------- 32.0/204.1 

In [8]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from dotenv import load_dotenv
import os

load_dotenv()
# Optional for safety/flexibility
# os.environ["HUGGINGFACE_HUB_TOKEN"] = os.getenv("HUGGINGFACE_HUB_TOKEN")
'''
may need to downgrade keras to 2.11.0
pip uninstall keras -y
pip install keras==2.11.0
'''
# ✅ This is all you need
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

vectorstore = FAISS.from_documents(docs, embeddings)
vectorstore.save_local("gandalf_index")

print("✅ Vectorstore saved.")


C:\Users\JonMa\AppData\Roaming\Python\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\JonMa\AppData\Roaming\Python\Python313\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\JonMa\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an adminis

✅ Vectorstore saved.


### 6. 🤖 Load a Local LLM (Option 1: Hugging Face)

In [ ]:
from langchain.llms import HuggingFaceHub
from langchain_huggingface import HuggingFaceEndpoint

llm = HuggingFaceEndpoint(
    repo_id="google/flan-t5-large",
    temperature=0.7,
    max_new_tokens=512,
    huggingfacehub_api_token=hf_token
)


C:\Users\JonMa\AppData\Local\Temp\ipykernel_42616\1459307106.py:3: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  llm = HuggingFaceHub(


# OPTIONAL: 6B 🤖 Load from Ollama (Option 2: Local Mistral/WizardLM)

In [9]:


# from langchain_community.llms import Ollama
# llm = Ollama(model="mistral")  # or wizardlm


In [12]:
### 7. 🔁 Ask Gandalf a Question (RAG Chain)

from langchain.chains import RetrievalQA

retriever = vectorstore.as_retriever()

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

question = "What happened in the mines of Moria?"
result = qa_chain(question)

print("🧙 Gandalf says:\n", result['result'])


C:\Users\JonMa\AppData\Local\Temp\ipykernel_42616\1316374485.py:14: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain(question)
C:\Users\JonMa\AppData\Roaming\Python\Python313\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


HfHubHTTPError: 503 Server Error: Service Temporarily Unavailable for url: https://router.huggingface.co/hf-inference/models/google/flan-t5-large